<h1>TIME SERIES ANALYSIS:<br>
<b>Forecasting the Annually Inflation Rate in the Philippines</b></h1>

<b>A. Importing Libraries and Data</b>

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
df = pd.read_csv('/content/Group 3 - Annually Inflation Rate in Philippines.csv')

FileNotFoundError: ignored

In [ ]:
df.head()

<b>B. Data Preprocessing</b>

In [ ]:
df['Year'] = pd.to_datetime(df['Year'],format='%Y')

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.drop(['Country','Indicator'], axis = 1)

In [ ]:
df1.head()

<b>C. Indexing with Data</b>

In [ ]:
df1 = df1.set_index('Year')
df1.index

In [ ]:
y = df1['Inflation Rate']

<b>D. Visualizing the Data</b>

In [ ]:
df1.plot(figsize=(15, 6))
plt.show()

<b>D.1 Visualizing using time-series decomposition </b>

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(df1, model='additive', freq=3)
fig = decomposition.plot()
plt.show()

<b>D.2 Visualizing using ACF </b>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df1, lags=10)

<b>D.3 Visualizing using PACF </b>

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df1, lags=10)

<b>D.4 Visualizing using Autocorrelation Plot </b>

In [ ]:
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df1)
pyplot.show()

<b>E. Evaluating the descriptive statistics of the data</b>

<b>Hypothesis Testing</b><br>
Ho: The time series data is not stationary<br>
Ha: The time series data is stationary

In [ ]:
df1.describe()

In [ ]:
values = df1.values
parts = int(len(values)/3)
part_1, part_2, part_3 = values[0:parts], values[parts:(
    parts*2)], values[(parts*2):(parts*3)]

mean_1, mean_2, mean_3 = part_1.mean(), part_2.mean(), part_3.mean()
var_1, var_2, var_3 = part_1.var(), part_2.var(), part_3.var()

print('mean1=%f, mean2=%f, mean3=%f' % (mean_1, mean_2, mean_3))
print('variance1=%f, variance2=%f, variance3=%f' % (var_1, var_2, var_3))

<b>E.1 Using Augmented Dickey-Fuller test</b>

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df1['Inflation Rate'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

<b>E.2 Using Adfuller Test</b>

In [ ]:
df1 = df.drop(['Country','Indicator'], axis = 1)
from statsmodels.tsa.stattools import adfuller
result = adfuller(df1['Inflation Rate'].dropna())
print('p-value :', result[1])
result = adfuller (df1['Inflation Rate'].diff().dropna())
print('p-value :', result[1])
result = adfuller(df1['Inflation Rate'].diff().diff().dropna())
print('p-value :', result[1])

Based on the descriptive statistics, the time series data provide a lower p-value of 0.06 in the ADF which signifies to reject the Null hypothesis. This indicates that the data is a stationary.

<b>F. Implementation of the ARIMA Model</b>

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
aic_results = []
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            aic_results.append(results.aic)
        except:
            continue



In [ ]:
df1 = pd.DataFrame(aic_results, columns=['aic'])
print(df1['aic'].min())

<b>G. Results of the Arima Model</b>

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 0, 1),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

<b>H. Descriptive Results of the ARIMA Model</b>

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

<b>I. Validation of the Forecast value in the ARIMA Model</b>

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2010-01-01'),dynamic=False)
pred_ci = pred.conf_int()
ax = y['2000-01-01':].plot(label='observed')

pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Year')
ax.set_ylabel('Inflation Rate')
plt.legend()
plt.show()

<b>J. Evaluation of the ARIMA Model</b>

<b>J.1 Evaluation using MSE</b>

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y['2010':]
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

<b>J.2 Evaluation using RMSE</b>

In [ ]:
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

<b>K. Forecasting the Inflation Rate after 6 years of Presidency of Newly Elect President Bong Bong Marcos </b>

In [ ]:
pred_uc = results.get_forecast(steps=8)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 7))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Year')
ax.set_ylabel('Inflation Rate')
plt.legend()
plt.show()

In [ ]:
print(pred_uc.predicted_mean)